In [1]:
# IMPORTS
import pandas as pd, numpy as np
import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score

In [2]:
# %load /home/adrian/Vadetis/vadetisweb/algorithms/rpca/loss.py
from abc import ABCMeta, abstractmethod
from numbers import Number

class MLoss:
    """
    An abstract class for a loss function suitable for an M-estimator solution using IRLS. Has to
    have a strict minimum at zero, be symmetric and positive definite, and increasing less than
    square.

    Programmatically, it has to implement a ``__call__(self, x)`` method, where `x` is a float, a
    ``diff(self, x)`` method returning its derivative at ``x``, and (optionally) a ``weight(self,
    x)=diff(self, x)/x``.
    """
    __metaclass__ = ABCMeta

    @abstractmethod
    def __call__(self, x):
        """
        Evaluates the function at x.

        Parameters
        ------------
        x : float
            Point to evaluate the function at.

        Returns
        ---------
        y : float
            Value at point x.
        """

    @abstractmethod
    def diff(self, x):
        """
        Evaluates the function's derivative at x.

        Parameters
        ------------
        x : float
            Point to evaluate the function derivative at.

        Returns
        ---------
        y : float
            Derivative value at point x.
        """

    def weight(self, x):
        """
        Evaluates the weight function (:=f(x)/x) induced by the loss function at x.

        Parameters
        ------------
        x : float
            Point to evaluate the weight function at.

        Returns
        ---------
        y : float
            Weight function value at point x.
        """
        return self.diff(x) / float(x)

class HuberLoss(MLoss):
    """
    Huber loss function: 
 
    .. math::
        f_\\delta (x) = \\begin{cases}
        \\frac{x^2}{2},\\,\\,\\,&\\vert x\\vert\\leq\\delta,\\\\
                \\delta\\left(\\vert x\\vert - \\frac{\\delta}{2} \\right),\\,\\,\\,&\\vert
        x\\vert\\geq\\delta
        \\end{cases}

    Parameters
    ------------
    delta : float >= 0
        Delta parameter in function definition.
    """
    def __init__(self, delta):
        assert isinstance(delta, Number)
        assert float(delta) >= 0, 'delta has to be non-negative.'

        self.delta = float(delta)
        self.delta_half_square = (self.delta ** 2) / 2.

    def __call__(self, x):
        x_flt = float(x)
        assert x_flt >= 0
        if x_flt <= self.delta:
            return (x_flt ** 2) / 2.
        else:
            return self.delta * x_flt - self.delta_half_square
    
    def diff(self, x):
        pass

    def weight(self, x):
        x_flt = float(x)
        assert x_flt >= 0
        if x_flt <= self.delta:
            return 1.
        else:
            return self.delta / x_flt


In [3]:
# %load /home/adrian/Vadetis/vadetisweb/algorithms/rpca/m_est_rpca.py
import warnings
from sklearn.decomposition import PCA
from sklearn.decomposition.base import _BasePCA
from sklearn.utils import check_array
from sklearn.utils.extmath import svd_flip
from scipy.sparse import issparse
from scipy import linalg
import numpy as np

class MRobustPCA(_BasePCA):
    """
    An implementation of Robust PCA using M-estimator loss. A particular case with the Huber loss
    function is described in the paper:

        B.T. Polyak, M.V. Khlebnikov: Principal Component Analysis: Robust Variants, 2017.

    Parameters
    -----------
    n_components: int or None
        Number of components. If None, n_components = data dimensionality.

    loss : MLossFunction object
        Loss function to be optimized during fitting. See "Loss functions" for details.

    model : string {'first', 'second'} (default 'first')
        Statistical model to be used during fitting, according to the original method. First is
        based on the iid noise assumption, second estimates the noise covariance structure but
        assumes a single cluster center for all points. For more details, see the original paper.abs

    eps : float >= 0, optional (default 1e-6)
        Max relative error for the objective function optimised by IRLS. Used as a stopping
        criterion.

    max_iter : int >= 0, optional (default 100)
        Max number of IRLS iterations performed during optimisation. Used as a stopping crietrion.

    Attributes
    ----------
    components\_ : array, [n_components, n_features]
        Principal axes in feature space. The components are sorted by
        ``explained_variance_``.

    explained_variance\_ : array, [n_components]
        The amount of variance explained by each of the selected components.

    explained_variance_ratio\_ : array, [n_components]
        Percentage of variance explained by each of the selected components.
        If ``n_components`` is not set then all components are stored and the
        sum of explained variances is equal to 1.0.

    mean\_ : array, [n_features]
        Per-feature empirical mean, estimated from the training set.
        Equal to `X.mean(axis=1)`.

    n_components\_ : int
        The number of components. It equals the parameter
        n_components, or n_features if n_components is None.    

    weights\_ : array, [n_samples]
        Weights assigned to input samples during IRLS optimisation.

    n_iterations\_ : int
        Number of iterations performed during fitting.

    errors\_ : array, [n_iterations\_]
        Errors achieved by the method on each iteration.
    """
    def __init__(self,
                 n_components,
                 loss,
                 model='first',
                 eps=1e-6,
                 max_iter=100):
        self.n_components = n_components
        self.loss = loss
        self.model = model
        self.eps = eps
        self.max_iter = max_iter
        self.whiten = False # TODO: implement proper whitening

    def fit(self, X, y=None, weights_init=None):
        """
        Fit the model with X.

        Parameters
        ----------
        X: array-like, shape (n_samples, n_features)
            Training data, where n_samples in the number of samples
            and n_features is the number of features.

        Returns
        -------
        self : object
            Returns the instance itself.
        """
        self._fit(X, weights_init)
        return self

    def fit_transform(self, X, y=None):
        """
        Fit to data, then transform it.

        Fits the model to X and returns a transformed version of X.
        
        Parameters
        ----------
        X: array-like, shape (n_samples, n_features)
            Training data, where n_samples in the number of samples
            and n_features is the number of features.

        Returns
        -------
        X_new: array-like, shape (n_samples, n_components)
            Transformed array.
        """
        return super(MRobustPCA, self).fit_transform(X, y)

    def _fit(self, X, weights_init=None):
        # Raise an error for sparse input.
        # This is more informative than the generic one raised by check_array.
        if issparse(X):
            raise TypeError('MRobustPCA does not support sparse input.')

        X = check_array(X, dtype=[np.float64], ensure_2d=True,
                        copy=True)

        # Handle n_components==None
        if self.n_components is None:
            n_components = X.shape[1]
        else:
            n_components = self.n_components

        return self._fit_model(X, n_components, weights_init)

    def _fit_model(self, X, n_components, weights_init):
        vectorized_loss = np.vectorize(self.loss.__call__)
        vectorized_weights = np.vectorize(self.loss.weight)

        n_samples, n_features = X.shape
        if weights_init is not None:
            self.weights_ = weights_init
        else:
            self.weights_ = 1. / n_samples * np.ones(n_samples)
        self.errors_ = [np.inf]
        self.n_iterations_ = 0
        not_done_yet = True
        while not_done_yet:
            # Calculating components with current weights
            self.mean_ = np.average(X, axis=0, weights=self.weights_)
            X_centered = X - self.mean_
            U, S, V = linalg.svd(X_centered * np.sqrt(self.weights_.reshape(-1,1)))
            # U, V = svd_flip(U, V)
            self.components_ = V[:n_components, :]

            # Calculate current errors in different models
            if self.model == 'first':
                non_projected_metric = np.eye(n_features) - \
                        self.components_.T.dot(self.components_)
                errors_raw = np.sqrt(np.diag(X_centered.dot(non_projected_metric.dot(X_centered.T))))
            elif self.model == 'second':
                # Obtain inverse empirical covariance from the SVD
                R_inv = np.diag(1. / S**2.)
                inverse_cov = V.T.dot(R_inv.dot(V))
                errors_raw = np.sqrt(np.diag(X_centered.dot(inverse_cov.dot(X_centered.T))))
            else:
                raise ValueError('Model should be either \"first\" or \"second\".') 

            errors_loss = vectorized_loss(errors_raw)

            # New weights based on errors
            self.weights_ = vectorized_weights(errors_raw)
            self.weights_ /= self.weights_.sum()
            # Checking stopping criteria
            self.n_iterations_ += 1
            old_total_error = self.errors_[-1]
            total_error = errors_loss.sum()

            if not np.equal(total_error, 0.):
                rel_error = abs(total_error - old_total_error) / abs(total_error)
            else:
                rel_error = 0.

            print('[RPCA] Iteraton %d: error %f, relative error %f'%(self.n_iterations_,
                                                                     total_error,
                                                                     rel_error))
            self.errors_.append(total_error)
            not_done_yet = rel_error > self.eps and self.n_iterations_ < self.max_iter
        if rel_error > self.eps:
            warnings.warn('[RPCA] Did not reach desired precision after %d iterations; relative\
                          error %f instead of specified maximum %f'%(self.n_iterations_,
                                                                     rel_error,
                                                                     self.eps))
        # Get variance explained by singular values
        explained_variance_ = (S ** 2) / n_samples
        total_var = explained_variance_.sum()
        if not np.equal(total_var, 0.):
            explained_variance_ratio_ = explained_variance_ / total_var
        else:
            explained_variance_ratio_ = np.zeros_like(explained_variance_)
        self.n_samples_, self.n_features_ = n_samples, n_features
        self.n_components_ = n_components
        self.explained_variance_ = explained_variance_[:n_components]
        self.explained_variance_ratio_ = \
            explained_variance_ratio_[:n_components]

        self.errors_ = np.array(self.errors_[1:])

    def transform(self, X):
        """Apply dimensionality reduction to X.

        X is projected on the first principal components previously extracted
        from a training set.

        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            New data, where n_samples is the number of samples
            and n_features is the number of features.

        Returns
        -------
        X_new : array-like, shape (n_samples, n_components)
        """
        return super(MRobustPCA, self).transform(X)

    def inverse_transform(self, X):
        """Transform data back to its original space.

        In other words, return an input X_original whose transform would be X.

        Parameters
        ----------
        X : array-like, shape (n_samples, n_components)
            New data, where n_samples is the number of samples
            and n_components is the number of components.

        Returns
        -------
        X_original array-like, shape (n_samples, n_features)

        Notes
        -----
        If whitening is enabled, inverse_transform will compute the
        exact inverse operation, which includes reversing whitening.
        """
        return super(MRobustPCA, self).inverse_transform(X)


/home/adrian/Vadetis/venv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.decomposition.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.decomposition. Anything that cannot be imported from sklearn.decomposition is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
# HELPER METHOD TO PRINT OUT VALUES CONTAINED IN RPCA MODEL
def print_rpca_model(model):
    """
    components\_ : array, [n_components, n_features]
        Principal axes in feature space. The components are sorted by
        ``explained_variance_``.

    explained_variance\_ : array, [n_components]
        The amount of variance explained by each of the selected components.

    explained_variance_ratio\_ : array, [n_components]
        Percentage of variance explained by each of the selected components.
        If ``n_components`` is not set then all components are stored and the
        sum of explained variances is equal to 1.0.

    mean\_ : array, [n_features]
        Per-feature empirical mean, estimated from the training set.
        Equal to `X.mean(axis=1)`.

    n_components\_ : int
        The number of components. It equals the parameter
        n_components, or n_features if n_components is None.    

    weights\_ : array, [n_samples]
        Weights assigned to input samples during IRLS optimisation.

    n_iterations\_ : int
        Number of iterations performed during fitting.

    errors\_ : array, [n_iterations\_]
        Errors achieved by the method on each iteration.
    """
    if hasattr(model, 'components_'):
        print("components: %s" % model.components_)

    if hasattr(model, 'explained_variance_'):
        print("explained_variance: %s" % model.explained_variance_)

    if hasattr(model, 'mean_'):
        print("mean: %s" % model.mean_)

    if hasattr(model, 'n_components_'):
        print("n_components: %d" % model.n_components_)

    if hasattr(model, 'weights_'):
        print("weights: %s" % model.weights_)

    if hasattr(model, 'n_iterations_'):
        print("n_iterations: %d" % model.n_iterations_)

    if hasattr(model, 'errors_'):
        print("errors: %s" % model.errors_)

In [5]:
# METHOD TO COMPUTE ERROR SCORE BETWEEN AN ORIGINAL AND A RECONSTRUCTED DATASET
def normalized_anomaly_scores(df_original, df_reconstructed):
    """
    The reconstruction error is the sum of the squared differences between the original and the reconstructed dataset.
    The sum of the squared differences is scaled by the max-min range of the sum of the squared differences,
    so that all reconstruction errors are within a range of 0 to 1 (normalized). In consequence, normal data
    should have a low score whereas anomalies have higher scores.

    :param df_original: the original dataset as dataframe
    :param df_reconstructed: the reconstructed dataset from rpca
    :return: anomaly scores as series with range 0 to 1
    """

    diff = np.sum((np.array(df_original) - np.array(df_reconstructed)) ** 2, axis=1)
    diff = pd.Series(data=diff, index=df_original.index)
    print("score before normalization")
    print(diff)
    diff = (diff - np.min(diff)) / (np.max(diff) - np.min(diff))
    print("\nscore after normalization")
    print(diff)
    print("\n")

    return diff

In [6]:
# HELPER METHODS TO COMPUTE PERFORMANCE METRICS

def get_threshold_scores(thresholds, y_scores, valid, upper_boundary=False):
    """
    Computes for each possible threshold the score for the performance metrics

    :param thresholds: a list of possible thresholds
    :param y_scores: the list of computed scores by the detection algorithm
    :param valid: the true class values to run the performance metric against
    :param upper_boundary: determines if score higher than thresholds are anomalies or not

    :return: array of scores for each threshold for each performance metric
    """
    scores = []
    for threshold in thresholds:
        y_hat = np.array(y_scores < threshold).astype(int) if upper_boundary == False else np.array(y_scores > threshold).astype(int)
        
        accuracy = accuracy_score(y_true=valid['class'].values, y_pred=y_hat)
        recall = recall_score(y_true=valid['class'].values, y_pred=y_hat)
        precision = precision_score(y_true=valid['class'].values, y_pred=y_hat)
        f1_score = fbeta_score(y_true=valid['class'].values, y_pred=y_hat, beta=1)
        print('Threshold: %.3f' % threshold)
        print('Accuracy Score: %.3f' % accuracy)
        print('Recall Score: %.3f' % recall)
        print('Precision Score: %.3f' % precision)
        print('F1 Score: %.3f' % f1_score)
        print("\n")
        scores.append([recall,
                       precision,
                       f1_score,
                       accuracy])

    return np.array(scores)

def get_max_score_index_for_score_type(threshold_scores, score_type):
    if score_type == "F1_SCORE":
        return threshold_scores[:, 2].argmax()
    elif score_type == "RECALL":
        return threshold_scores[:, 0].argmax()
    elif score_type == "PRECISION":
        return threshold_scores[:, 1].argmax()
    elif score_type == "ACCURACY":
        return threshold_scores[:, 3].argmax()
    raise ValueError

In [7]:
# MAIN DATA
date_now = dt.datetime.now()
date_today = date_now.replace(hour=0, minute=0, second=0, microsecond=0)
days = pd.date_range(date_today, date_today + dt.timedelta(1), freq='D')

np.random.seed(seed=1234)
data_1 = np.random.randint(1, high=10, size=len(days))
data_2 = np.random.randint(1, high=10, size=len(days))
data_3 = np.array([100, 2])
df = pd.DataFrame({'time': days, 'feature_1': data_1, 'feature_2': data_2, 'feature_3': data_3 })
df = df.set_index('time')

print(df)

            feature_1  feature_2  feature_3
time                                       
2020-05-13          4          6        100
2020-05-14          7          5          2


In [8]:
# MAIN DATA CLASS
"""
    Row with index 0 in df is outlier, as it contains a high value out of bound (1, 10)
"""
data_df_class = np.array([True, False])
df_class = pd.DataFrame({'time': days, 'class': data_df_class })
df_class = df_class.set_index('time')

print(df_class)

            class
time             
2020-05-13   True
2020-05-14  False


In [9]:
# TRAINING DATA
date_now = dt.datetime.now()
date_today = date_now.replace(hour=0, minute=0, second=0, microsecond=0)
days = pd.date_range(date_today, date_today + dt.timedelta(3), freq='D')

np.random.seed(seed=4321)
data_1 = np.random.randint(1, high=10, size=len(days))
data_2 = np.array([3, 100, 6, 100])
data_3 = np.random.randint(1, high=10, size=len(days))
df_train = pd.DataFrame({'time': days, 'feature_1': data_1, 'feature_2': data_2, 'feature_3': data_3 })
df_train = df_train.set_index('time')

print(df_train)

            feature_1  feature_2  feature_3
time                                       
2020-05-13          3          3          8
2020-05-14          9        100          6
2020-05-15          3          6          3
2020-05-16          2        100          5


In [10]:
# TRAINING DATA CLASS
"""
    Rows with index 1 and 3 in df_train are outliers, as they contain a high value out of bound (1, 10)
"""
data_train_class = np.array([False, True, False, True])
df_train_class = pd.DataFrame({'time': days, 'class': data_train_class })
df_train_class = df_train_class.set_index('time')

print(df_train_class)

            class
time             
2020-05-13  False
2020-05-14   True
2020-05-15  False
2020-05-16   True


In [11]:
# TRAINING DATA SPLIT INTO TRAIN AND VALIDATION PARTS
"""
    stratify parameter makes a split so that the proportion of values in the sample produced
    will be the same as the proportion of values provided to parameter stratify
"""
X_train, X_test, y_train, y_test = train_test_split(df_train, df_train_class, train_size=0.5, random_state=10, stratify=df_train_class)

print("=== X_TRAIN ===")
print(X_train)
print("\n")

print("=== y_train ===")
print(y_train)
print("\n")

print("=== X_test ===")
print(X_test)
print("\n")

print("=== y_test ===")
print(y_test)
print("\n")

=== X_TRAIN ===
            feature_1  feature_2  feature_3
time                                       
2020-05-14          9        100          6
2020-05-13          3          3          8


=== y_train ===
            class
time             
2020-05-14   True
2020-05-13  False


=== X_test ===
            feature_1  feature_2  feature_3
time                                       
2020-05-15          3          6          3
2020-05-16          2        100          5


=== y_test ===
            class
time             
2020-05-15  False
2020-05-16   True




In [12]:
# DEFINE MODEL
"""
    Dimensionality reduction using Robust PCA and Huber Loss Function
    
    :param delta: delta value (=threshold) for Huber Loss Function
    :param n_components: the number of principal components for dimensionality reduction
"""
huber_loss = HuberLoss(delta=1)
M_rpca = MRobustPCA(1, huber_loss)

# We see that M_rpca is only initialized, but hasn't computed anything yet
print_rpca_model(M_rpca)

In [13]:
# FIT MODEL ON TRAINING (X_train)
M_rpca.fit(X_train)

print_rpca_model(M_rpca)

[RPCA] Iteraton 1: error 0.000000, relative error inf
[RPCA] Iteraton 2: error 0.000000, relative error 0.000000
components: [[-0.06172461 -0.99788113  0.02057487]]
explained_variance: [1181.125]
mean: [ 6.  51.5  7. ]
n_components: 1
weights: [0.5 0.5]
n_iterations: 2
errors: [2.50893924e-13 2.50893924e-13]


In [14]:
# REDUCE TEST SET WITH FITTED MODEL
X_test_reduced = M_rpca.transform(X_test)
X_test_reduced = pd.DataFrame(data=X_test_reduced, index=X_test.index)

print("=== X_test_reduced ===")
print(X_test_reduced)

# NOTE THE MODEL HAS STILL SAME COMPONENTS AND ATTRIBUTES
print("\nRPCA MODEL")
print_rpca_model(M_rpca)

=== X_test_reduced ===
                    0
time                 
2020-05-15  45.506466
2020-05-16 -48.191486

RPCA MODEL
components: [[-0.06172461 -0.99788113  0.02057487]]
explained_variance: [1181.125]
mean: [ 6.  51.5  7. ]
n_components: 1
weights: [0.5 0.5]
n_iterations: 2
errors: [2.50893924e-13 2.50893924e-13]


In [15]:
# RECONSTRUCT TEST SET WITH FITTED MODEL
X_test_reconstructed = M_rpca.inverse_transform(X_test_reduced)
X_test_reconstructed = pd.DataFrame(data=X_test_reconstructed, index=X_test.index)

print("=== X_test_reconstructed ===")
print(X_test_reconstructed)
print("\n=== X_test - original for reference - ===")
print(X_test)

# NOTE THE MODEL HAS STILL SAME COMPONENTS AND ATTRIBUTES
print("\nRPCA MODEL")
print_rpca_model(M_rpca)

=== X_test_reconstructed ===
                   0          1         2
time                                     
2020-05-15  3.191131   6.089957  7.936290
2020-05-16  8.974600  99.589375  6.008467

=== X_test - original for reference - ===
            feature_1  feature_2  feature_3
time                                       
2020-05-15          3          6          3
2020-05-16          2        100          5

RPCA MODEL
components: [[-0.06172461 -0.99788113  0.02057487]]
explained_variance: [1181.125]
mean: [ 6.  51.5  7. ]
n_components: 1
weights: [0.5 0.5]
n_iterations: 2
errors: [2.50893924e-13 2.50893924e-13]


In [16]:
# COMPUTE SCORE FROM RECONSTRUCTION ERROR
"""
    The larger error score the more likley it's an outlier
"""
y_test_scores = normalized_anomaly_scores(X_test, X_test_reconstructed)
print(y_test_scores)

# COMBINE WITH TRUTH
y_test_scores_class = y_test_scores.to_frame().join(y_test)
print(y_test_scores_class)

score before normalization
time
2020-05-15    24.411578
2020-05-16    49.830670
dtype: float64

score after normalization
time
2020-05-15    0.0
2020-05-16    1.0
dtype: float64


time
2020-05-15    0.0
2020-05-16    1.0
dtype: float64
              0  class
time                  
2020-05-15  0.0  False
2020-05-16  1.0   True


In [17]:
# COMPUTE ANOMALY SCORE THRESHOLD
"""
    we want to determine an threshold for anomaly detection. If a computed score is above the threshold 
    it will be marked as anomaly, otherwise it will marked as normal data
    
    for simplicity we use range (0,1) and only 4 candidates
"""
lower_bound = 0
higher_bound = 1
thresholds = np.linspace(lower_bound, higher_bound, 4) # => 4 candidates

print("possible anomaly score thresholds")
print(thresholds)

possible anomaly score thresholds
[0.         0.33333333 0.66666667 1.        ]


In [18]:
# For all possible thresholds we compute the performance metrics, 
# and select the final threshold for maximizing F1 Score
threshold_scores = get_threshold_scores(thresholds, y_test_scores, y_test, upper_boundary=True)
selected_index = get_max_score_index_for_score_type(threshold_scores, "F1_SCORE")
selected_threshold = thresholds[selected_index]

# Now we have our final threshold 0.0, which means anything with score > 0.0 will be marked as anomaly
print(selected_threshold)

Threshold: 0.000
Accuracy Score: 1.000
Recall Score: 1.000
Precision Score: 1.000
F1 Score: 1.000


Threshold: 0.333
Accuracy Score: 1.000
Recall Score: 1.000
Precision Score: 1.000
F1 Score: 1.000


Threshold: 0.667
Accuracy Score: 1.000
Recall Score: 1.000
Precision Score: 1.000
F1 Score: 1.000


Threshold: 1.000
Accuracy Score: 0.500
Recall Score: 0.000
Precision Score: 0.000
F1 Score: 0.000


0.0


/home/adrian/Vadetis/venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
# NOW LETS COMPUTE ANOMALY SCORES FOR OUR MAIN DATASET
# NOTE THE MODEL HAS STILL SAME COMPONENTS AND ATTRIBUTES
print(df)
print("\nRPCA MODEL")
print_rpca_model(M_rpca)

            feature_1  feature_2  feature_3
time                                       
2020-05-13          4          6        100
2020-05-14          7          5          2

RPCA MODEL
components: [[-0.06172461 -0.99788113  0.02057487]]
explained_variance: [1181.125]
mean: [ 6.  51.5  7. ]
n_components: 1
weights: [0.5 0.5]
n_iterations: 2
errors: [2.50893924e-13 2.50893924e-13]


In [20]:
# REDUCE MAIN DATASET WITH FITTED MODEL
X_df_reduced = M_rpca.transform(df)
X_df_reduced = pd.DataFrame(data=X_df_reduced, index=df.index)

print("=== X_df_reduced ===")
print(X_df_reduced)

# NOTE THE MODEL HAS STILL SAME COMPONENTS AND ATTRIBUTES
print("\nRPCA MODEL")
print_rpca_model(M_rpca)

=== X_df_reduced ===
                    0
time                 
2020-05-13  47.440503
2020-05-14  46.236874

RPCA MODEL
components: [[-0.06172461 -0.99788113  0.02057487]]
explained_variance: [1181.125]
mean: [ 6.  51.5  7. ]
n_components: 1
weights: [0.5 0.5]
n_iterations: 2
errors: [2.50893924e-13 2.50893924e-13]


In [21]:
# RECONSTRUCT MAIN SET WITH FITTED MODEL
X_df_reconstructed = M_rpca.inverse_transform(X_df_reduced)
X_df_reconstructed = pd.DataFrame(data=X_df_reconstructed, index=df.index)

print("=== X_df_reconstructed ===")
print(X_df_reconstructed)
print("\n=== df - original for reference - ===")
print(df)

# NOTE THE MODEL HAS STILL SAME COMPONENTS AND ATTRIBUTES
print("\nRPCA MODEL")
print_rpca_model(M_rpca)

=== X_df_reconstructed ===
                   0         1         2
time                                    
2020-05-13  3.071754  4.160017  7.976082
2020-05-14  3.146047  5.361096  7.951318

=== df - original for reference - ===
            feature_1  feature_2  feature_3
time                                       
2020-05-13          4          6        100
2020-05-14          7          5          2

RPCA MODEL
components: [[-0.06172461 -0.99788113  0.02057487]]
explained_variance: [1181.125]
mean: [ 6.  51.5  7. ]
n_components: 1
weights: [0.5 0.5]
n_iterations: 2
errors: [2.50893924e-13 2.50893924e-13]


In [22]:
# COMPUTE ANOMALY SCORE 
scores = normalized_anomaly_scores(df, X_df_reconstructed)
print("scores")
print(scores)

y_hat_results = (scores > selected_threshold).astype(int)
print("\ndetection results")
print(y_hat_results)

# COMPARE TO TRUTH
y_truth = df_class.values.astype(int)
print("\nComparision - Detection <-> Truth")
print(y_hat_results.to_frame().join(df_class))

accuracy = accuracy_score(y_pred=y_hat_results, y_true=y_truth)
recall = recall_score(y_pred=y_hat_results, y_true=y_truth)
precision = precision_score(y_pred=y_hat_results, y_true=y_truth)
f1_score = fbeta_score(y_pred=y_hat_results, y_true=y_truth, beta=1)

print('\nThreshold: %.3f' % selected_threshold)
print('Accuracy Score: %.3f' % accuracy)
print('Recall Score: %.3f' % recall)
print('Precision Score: %.3f' % precision)
print('F1 Score: %.3f' % f1_score)

score before normalization
time
2020-05-13    8472.648640
2020-05-14      50.401524
dtype: float64

score after normalization
time
2020-05-13    1.0
2020-05-14    0.0
dtype: float64


scores
time
2020-05-13    1.0
2020-05-14    0.0
dtype: float64

detection results
time
2020-05-13    1
2020-05-14    0
dtype: int64

Comparision - Detection <-> Truth
            0  class
time                
2020-05-13  1   True
2020-05-14  0  False

Threshold: 0.000
Accuracy Score: 1.000
Recall Score: 1.000
Precision Score: 1.000
F1 Score: 1.000
